In [85]:
# Import libaries
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder
import warnings
warnings.filterwarnings('ignore')

In [86]:
# Load the data set 
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [87]:
# Dealing with missing values
df['age'].fillna(df['age'].median() , inplace=True)
df['embarked'].fillna(df['embarked'].mode() , inplace=True)
df['embark_town'].fillna(df['embark_town'].mode() , inplace=True)

# Drop the deck column
df.drop('deck' , axis=1 ,inplace=True)

In [88]:
# encode all categorical variables and object variables

le_sex = LabelEncoder()
le_embarked = LabelEncoder()
le_class = LabelEncoder()
le_who = LabelEncoder()
le_embark_town = LabelEncoder()
le_alive = LabelEncoder()

df['sex'] = le_sex.fit_transform(df['sex'])
df['embarked'] = le_embarked.fit_transform(df['embarked'])
df['class'] = le_class.fit_transform(df['class'])
df['who'] = le_who.fit_transform(df['who'])
df['embark_town'] = le_embark_town.fit_transform(df['embark_town'])
df['alive'] = le_alive.fit_transform(df['alive'])
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,1,22.0,1,0,7.2500,2,2,1,True,2,0,False
1,1,1,0,38.0,1,0,71.2833,0,0,2,False,0,1,False
2,1,3,0,26.0,0,0,7.9250,2,2,2,False,2,1,True
3,1,1,0,35.0,1,0,53.1000,2,0,2,False,2,1,False
4,0,3,1,35.0,0,0,8.0500,2,2,1,True,2,0,True


In [89]:
# inverse transform the data

df['sex'] = le_sex.inverse_transform(df['sex'])
df['embarked'] = le_embarked.inverse_transform(df['embarked'])
df['class'] = le_class.inverse_transform(df['class'])
df['who'] = le_who.inverse_transform(df['who'])
df['embark_town'] = le_embark_town.inverse_transform(df['embark_town'])
df['alive'] = le_alive.inverse_transform(df['alive'])
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True


## ONE HOT Encoding

In [90]:
from sklearn.preprocessing import OneHotEncoder

df = sns.load_dataset('titanic')

cat_columns = ['sex', 'embarked']



In [91]:
encoder = OneHotEncoder(sparse=False)
encoded_df = pd.DataFrame(encoder.fit_transform(df[cat_columns]))

# concatenate the dataframes 
df = pd.concat([df, encoded_df], axis=1)
# df.drop(cat_columns, axis=1, inplace=True)
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,...,deck,embark_town,alive,alone,0,1,2,3,4,5
0,0,3,male,22.0,1,0,7.2500,S,Third,man,...,NaN,Southampton,no,False,0.0,1.0,0.0,0.0,1.0,0.0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,...,C,Cherbourg,yes,False,1.0,0.0,1.0,0.0,0.0,0.0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,...,NaN,Southampton,yes,True,1.0,0.0,0.0,0.0,1.0,0.0
3,1,1,female,35.0,1,0,53.1000,S,First,woman,...,C,Southampton,yes,False,1.0,0.0,0.0,0.0,1.0,0.0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,...,NaN,Southampton,no,True,0.0,1.0,0.0,0.0,1.0,0.0


In [92]:
# We need to extract the original categories for each encoded column
original_categories = {col: encoder.categories_[i] for i, col in enumerate(cat_columns)}
# Manual creation of feature names
feature_names = []
for i, col in enumerate(cat_columns):
    for category in encoder.categories_[i]:
        feature_names.append(f"{col}_{category}")

encoded_df = pd.DataFrame(encoded_df, columns=feature_names)

df.head()


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,...,deck,embark_town,alive,alone,0,1,2,3,4,5
0,0,3,male,22.0,1,0,7.2500,S,Third,man,...,NaN,Southampton,no,False,0.0,1.0,0.0,0.0,1.0,0.0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,...,C,Cherbourg,yes,False,1.0,0.0,1.0,0.0,0.0,0.0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,...,NaN,Southampton,yes,True,1.0,0.0,0.0,0.0,1.0,0.0
3,1,1,female,35.0,1,0,53.1000,S,First,woman,...,C,Southampton,yes,False,1.0,0.0,0.0,0.0,1.0,0.0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,...,NaN,Southampton,no,True,0.0,1.0,0.0,0.0,1.0,0.0


In [93]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,...,deck,embark_town,alive,alone,0,1,2,3,4,5
0,0,3,male,22.0,1,0,7.2500,S,Third,man,...,NaN,Southampton,no,False,0.0,1.0,0.0,0.0,1.0,0.0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,...,C,Cherbourg,yes,False,1.0,0.0,1.0,0.0,0.0,0.0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,...,NaN,Southampton,yes,True,1.0,0.0,0.0,0.0,1.0,0.0
3,1,1,female,35.0,1,0,53.1000,S,First,woman,...,C,Southampton,yes,False,1.0,0.0,0.0,0.0,1.0,0.0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,...,NaN,Southampton,no,True,0.0,1.0,0.0,0.0,1.0,0.0


In [94]:
ohe_sex = OneHotEncoder()
ohe_embarked = OneHotEncoder()
ohe_class = OneHotEncoder()
ohe_who = OneHotEncoder()
ohe_embark_town = OneHotEncoder()
ohe_alive = OneHotEncoder()


df['embarked'] = ohe_embarked.fit_transform(df[['embarked']]).toarray()
df['class'] = ohe_class.fit_transform(df[['class']]).toarray()
df.head()


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,...,deck,embark_town,alive,alone,0,1,2,3,4,5
0,0,3,male,22.0,1,0,7.2500,0.0,0.0,man,...,NaN,Southampton,no,False,0.0,1.0,0.0,0.0,1.0,0.0
1,1,1,female,38.0,1,0,71.2833,1.0,1.0,woman,...,C,Cherbourg,yes,False,1.0,0.0,1.0,0.0,0.0,0.0
2,1,3,female,26.0,0,0,7.9250,0.0,0.0,woman,...,NaN,Southampton,yes,True,1.0,0.0,0.0,0.0,1.0,0.0
3,1,1,female,35.0,1,0,53.1000,0.0,1.0,woman,...,C,Southampton,yes,False,1.0,0.0,0.0,0.0,1.0,0.0
4,0,3,male,35.0,0,0,8.0500,0.0,0.0,man,...,NaN,Southampton,no,True,0.0,1.0,0.0,0.0,1.0,0.0


In [95]:
# inverse the embarked and class feature


## Oridnal encoding

In [96]:
OE_sex = OrdinalEncoder() 

# Fit the sex columns
df['sex'] = OE_sex.fit_transform(df[['sex']])

# Create dataframe containg encode
Encode_sex = pd.DataFrame(df['sex'])

In [97]:
Encode_sex.head(3)

,sex
0,1.0
1,0.0
2,0.0


In [98]:
# Decode the data
Decode_sex = OE_sex.inverse_transform(Encode_sex)
Decode_sex

array([['male'],
       ['female'],
       ['female'],
       ['female'],
       ['male'],
       ['male'],
       ['male'],
       ['male'],
       ['female'],
       ['female'],
       ['female'],
       ['female'],
       ['male'],
       ['male'],
       ['female'],
       ['female'],
       ['male'],
       ['male'],
       ['female'],
       ['female'],
       ['male'],
       ['male'],
       ['female'],
       ['male'],
       ['female'],
       ['female'],
       ['male'],
       ['male'],
       ['female'],
       ['male'],
       ['male'],
       ['female'],
       ['female'],
       ['male'],
       ['male'],
       ['male'],
       ['male'],
       ['male'],
       ['female'],
       ['female'],
       ['female'],
       ['female'],
       ['male'],
       ['female'],
       ['female'],
       ['male'],
       ['male'],
       ['female'],
       ['male'],
       ['female'],
       ['male'],
       ['male'],
       ['female'],
       ['female'],
       ['male'],
       ['ma